In [1]:
import boto3
from config import *

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='ap-south-1'
)

ec2=session.resource('ec2')


In [2]:
def launch_t2micro_Amazon_Linux():

    instance=ec2.create_instances(
        ImageId="ami-0ded8326293d3201b", # amzn linux 2
        InstanceType='t2.micro',
        MinCount=1,
        MaxCount=1,
        SecurityGroups=['launch-wizard-1']
    )[0]

    instance.create_tags(Tags=[{'Key': 'Name', 'Value': 'AmazonLinux'}])

    print(instance)
    print('Running and waiting for them to be in running state')
    instance.wait_until_running()

    return instance

In [3]:
launch_t2micro_Amazon_Linux()

ec2.Instance(id='i-02393af0e3314f371')
Running and waiting for them to be in running state


ec2.Instance(id='i-02393af0e3314f371')

In [4]:
def launch_t2nano_ubuntu():
   
    instance = ec2.create_instances(
        ImageId='ami-0f5ee92e2d63afc18',
        InstanceType='t2.micro',
        MinCount=1,
        MaxCount=1,
        SecurityGroups=['launch-wizard-1'],
        KeyName='CS351_KEY',
        UserData=open('startup_script.sh', 'r').read(),
    )
    instance_1=instance[0].create_tags(Tags=[{'Key': 'Name', 'Value': 'UbuntuInstance11'}])
    
    print(instance_1,'\n',instance_1)
    print('Running and waiting for them to be in running state')
    instance[0].wait_until_running()
    return instance



In [5]:
launch_t2nano_ubuntu()

[ec2.Tag(resource_id='i-00cfc3aaf647edb56', key='Name', value='UbuntuInstance11')] 
 [ec2.Tag(resource_id='i-00cfc3aaf647edb56', key='Name', value='UbuntuInstance11')]
Running and waiting for them to be in running state


[ec2.Instance(id='i-00cfc3aaf647edb56')]

In [25]:
def list_running_instances():

    running_instances = ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

    for instance in running_instances:
        print(f"Instance ID: {instance.id}, State: {instance.state['Name']}")


In [26]:
list_running_instances()

Instance ID: i-03d4c81664ea68fa9, State: running
Instance ID: i-0cbf6021e3178839d, State: running
Instance ID: i-0d9ccb10f1eccdf6d, State: running
Instance ID: i-0917eeb4b26c5a438, State: running
Instance ID: i-04945b87c5999aa06, State: running


In [27]:
def instances_health():
    ec2_client=session.client('ec2')
    running_instances = ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['running']}]);

    for instance in running_instances:

        print(instance)
        
        response = ec2_client.describe_instance_status(
            InstanceIds=[instance.instance_id]
        )


        if len(response['InstanceStatuses']) > 0:
            status = response['InstanceStatuses'][0]['InstanceStatus']
            print(f"Instance {instance.instance_id} status\n{status['Details'][0]['Name']} --> {status['Details'][0]['Status']}\nHealth Status --> {status['Status']}\n")
        else:
            print(f"Instance {instance.instance_id} not found or status information not available.")


In [28]:
instances_health()

ec2.Instance(id='i-03d4c81664ea68fa9')
Instance i-03d4c81664ea68fa9 status
reachability --> passed
Health Status --> ok

ec2.Instance(id='i-0cbf6021e3178839d')
Instance i-0cbf6021e3178839d status
reachability --> initializing
Health Status --> initializing

ec2.Instance(id='i-0d9ccb10f1eccdf6d')
Instance i-0d9ccb10f1eccdf6d status
reachability --> passed
Health Status --> ok

ec2.Instance(id='i-0917eeb4b26c5a438')
Instance i-0917eeb4b26c5a438 status
reachability --> passed
Health Status --> ok

ec2.Instance(id='i-04945b87c5999aa06')
Instance i-04945b87c5999aa06 status
reachability --> initializing
Health Status --> initializing



In [31]:
def host_server_instance():


    instance = ec2.create_instances(
        ImageId='ami-0ded8326293d3201b',  
        InstanceType='t2.micro',
        MinCount=1,
        MaxCount=1,
        SecurityGroups=['launch-wizard-1'],  
        UserData=open('startup_script.sh', 'r').read(),
    )

    instance[0].create_tags(Tags=[{'Key': 'Name', 'Value': 'Host Server'}])

    print('Running and waiting for DNS')
    instance[0].wait_until_running()


    public_dns = instance[0].public_dns_name
    print("Public DNS:", public_dns)


In [32]:
host_server_instance()

Running and waiting for DNS
Public DNS: ec2-13-233-166-127.ap-south-1.compute.amazonaws.com


In [33]:
def stop_instances():

    running_instances = ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

    for instance in running_instances:
        print(instance.id +" Stopped ")
        instance.stop();

In [34]:
stop_instances()

i-01e53ede19bdc9b57 Stopped 
i-03d4c81664ea68fa9 Stopped 
i-0cbf6021e3178839d Stopped 
i-0d81567eaac85ebbf Stopped 
i-0d9ccb10f1eccdf6d Stopped 
i-0917eeb4b26c5a438 Stopped 
i-04945b87c5999aa06 Stopped 


In [8]:
def terminate_instance():

    running_instances = ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

    for instance in running_instances:
        print (instance.id + " Terminated")
        instance.terminate()



In [ ]:
terminate_instance()